In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import GRU
import re

In [3]:
df=pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [4]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
y=le.fit_transform(df['sentiment'])

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [5]:
from sklearn.model_selection import train_test_split
df_train, df_test,y_train,y_test= train_test_split(df,y,test_size = 0.20, random_state = 0)

In [6]:
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

#Removes Emojis
def remove_emoji(data):
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    data=emoji_clean.sub(r'',data)
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

df_train['review'] = df_train['review'].apply(lambda z: remove_punctuations(z))
df_train['review'] = df_train['review'].apply(lambda z: remove_html(z))
df_train['review'] = df_train['review'].apply(lambda z: remove_url(z))
df_train['review'] = df_train['review'].apply(lambda z: remove_emoji(z))

df_test['review'] = df_test['review'].apply(lambda z: remove_punctuations(z))
df_test['review'] = df_test['review'].apply(lambda z: remove_html(z))
df_test['review'] = df_test['review'].apply(lambda z: remove_url(z))
df_test['review'] = df_test['review'].apply(lambda z: remove_emoji(z))

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [11]:
# Tokenize text data
MAX_LEN = 128
X_train_tokenized = tokenizer.batch_encode_plus(
    df_train['review'].tolist(),
    max_length=MAX_LEN,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_token_type_ids=False
)
X_test_tokenized = tokenizer.batch_encode_plus(
    df_test['review'].tolist(),
    max_length=MAX_LEN,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_token_type_ids=False
)

In [12]:
attention_masks = np.array(X_train_tokenized['attention_mask'])
X_train = np.array(X_train_tokenized['input_ids'])
X_test = np.array(X_test_tokenized['input_ids'])

In [14]:
input_layer = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")

# Connect tokenizer output to BERT model
bert_output = bert_model(input_layer)[0]

# Define LSTM layer
reshape_layer = tf.keras.layers.Reshape((1, -1))(bert_output)
lstm_layer = Bidirectional(LSTM(128, return_sequences=True))(reshape_layer)

# Define pooling layer
pooling_layer = GlobalMaxPooling1D()(lstm_layer)

# Define output layer
output_layer = Dense(units=1, activation='sigmoid')(pooling_layer)

# Define the model
model_1 = Model(inputs=[input_layer], outputs=[output_layer])

# Compile the model
model_1.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'])

In [19]:
history = model_1.fit(
    x=X_train,
    y=y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=16,
    callbacks=[EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)]
)

Epoch 1/5
2000/2000 [==============================] - 566s 283ms/step - loss: 0.0126 - accuracy: 0.9965 - val_loss: 0.5110 - val_accuracy: 0.8926
Epoch 2/5
2000/2000 [==============================] - 565s 282ms/step - loss: 0.0125 - accuracy: 0.9961 - val_loss: 0.4839 - val_accuracy: 0.8969
Epoch 3/5
2000/2000 [==============================] - 565s 282ms/step - loss: 0.0084 - accuracy: 0.9975 - val_loss: 0.4632 - val_accuracy: 0.8970
Epoch 4/5
2000/2000 [==============================] - 565s 283ms/step - loss: 0.0082 - accuracy: 0.9973 - val_loss: 0.5045 - val_accuracy: 0.8972


In [20]:
batch_size = 16  # Set batch size to the same value as used in training
loss, accuracy = model_1.evaluate(X_test, y_test, batch_size=batch_size)
print('Test accuracy:', accuracy)

625/625 [==============================] - 54s 87ms/step - loss: 0.4845 - accuracy: 0.8955
Test accuracy: 0.8955000042915344
